# 1. load/reload all dependency

In [25]:
import numpy as np
import pandas as pd
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import sys
sys.path.append("../")
import pyhealth.datasets.datasets as datasets
import pyhealth.models.models as models
from importlib import reload
reload(datasets)
reload(models)
import torch
print(torch.cuda.is_available())


True


# 2. load dataset and initialize model

In [35]:
load MIMIC-III
dataset = datasets.MIMIC_III(
    table_names=['med', 'prod', 'diag']
)

# initialize the model and build the dataloaders
dataset.get_dataloader("GAMENet")
model = models.GAMENet(
    voc_size=dataset.voc_size,
    ehr_adj=dataset.ehr_adj,
    ddi_adj=dataset.ddi_adj,
    emb_dim=64,
    ddi_in_memory=True,
)

generated train/val/test dataloaders for GAMENet model!


# 3. train and test

In [ ]:
# set trainer with checkpoint
checkpoint_callback = ModelCheckpoint(dirpath='./model_cpt')
trainer = Trainer(
    gpus=1,
    max_epochs=5,
    progress_bar_refresh_rate=5,
    callbacks=[checkpoint_callback],
)

# train model
trainer.fit(
    model=model,
    train_dataloaders=dataset.train_loader,
    val_dataloaders=dataset.val_loader,
)

# test the best model
model.summary(
    test_dataloaders=dataset.test_loader,
    ckpt_path=checkpoint_callback.best_model_path,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name       | Type       | Params
------------------------------------------
0 | embeddings | ModuleList | 551 K 
1 | dropout    | Dropout    | 0     
2 | encoders   | ModuleList | 49.9 K
3 | query      | Sequential | 8.3 K 
4 | ehr_gcn    | GCN        | 41.8 K
5 | ddi_gcn    | GCN        | 41.8 K
6 | output     | Sequential | 40.4 K
------------------------------------------
673 K     Trainable params
74.4 K    Non-trainable params
748 K     Total params
2.993     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]